In [2]:
from src.combined_processor import CombinedProcessor
from src.eeg_config import EEGConfig
from run.train_VQVAE import train_VQVAE
from src.utils import *

from torch.utils.data import DataLoader, random_split
from moabb.datasets import Lee2019_SSVEP
import lightning.pytorch as pl
import pandas as pd

<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
2025-01-16 14:21:31.444970: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 14:21:31.474133: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the follow

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/opt/pythonenvs/class_fall2023/lib/python3.8/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [4]:
lee_config = EEGConfig(
    dataset=Lee2019_SSVEP(),
    num_channels=62,
    sample_rate=1000,
    sample_duration=4,
    subject_range=(1, 2),
    filter_range=(5, 95),
    channel_range=("Fp1", "PO4"),
)

combined_processor = CombinedProcessor(configs=[lee_config])

TimeoutError: [Errno 110] Connection timed out

In [4]:
train_set, val_set = random_split(combined_processor, [.8, .2])
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)

In [ ]:
model = train_VQVAE(
    dl_train = train_loader,
    dl_val = val_loader,
    epochs = epochs,
    train_bool = True,
    eff_net_PATH = eff_net_PATH,
    classes = [],
    in_channels = lee_config.num_channels)

In [2]:
test_set = Lee2019_SSVEP()
subjects_data = test_set.get_data(subjects=[1])
for subject, data in subjects_data.items():
    for run in data.items():
        for trial in run[1].items():
            hello = pd.DataFrame(run[1][trial[0]].to_data_frame())
            print(hello)

            print(run[1][trial[0]].info['ch_names'])  # List of channel names
            print(run[1][trial[0]].info['sfreq'])  # Sampling frequency
            print(run[1][trial[0]].info['nchan'])  # Number of channels

            # View all metadata
            print(run[1][trial[0]].info)


             time         Fp1         Fp2          F7          F3          Fz  \
0           0.000   42.400002   28.700001    2.900000   35.000000  156.100006   
1           0.001   46.099998   40.099998    9.300000   39.799999  157.500000   
2           0.002   43.000000   40.000000    9.500000   36.299999  153.899994   
3           0.003   40.900002   35.000000    8.100000   35.599998  152.600006   
4           0.004   41.700001   39.500000   10.600000   40.700001  153.899994   
...           ...         ...         ...         ...         ...         ...   
1560075  1560.075  387.700012  439.100006  235.500000  254.100006  252.600006   
1560076  1560.076  382.899994  436.700012  228.199997  249.800003  250.300003   
1560077  1560.077  380.100006  430.399994  218.800003  243.600006  245.000000   
1560078  1560.078  394.399994  437.100006  234.399994  255.100006  257.299988   
1560079  1560.079  405.200012  452.299988  241.000000  263.000000  268.200012   

                 F4        